# Character level language model - Dinosaurus land

Welcome to Dinosaurus Island! 65 million years ago, dinosaurs existed, and in this assignment they are back. You are in charge of a special task. Leading biology researchers are creating new breeds of dinosaurs and bringing them to life on earth, and your job is to give names to these dinosaurs. If a dinosaur does not like its name, it might go beserk, so choose wisely! 

<table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">

</td>

</table>

Luckily you have learned some deep learning and you will use it to save the day. Your assistant has collected a list of all the dinosaur names they could find, and compiled them into this [dataset](dinos.txt). (Feel free to take a look by clicking the previous link.) To create new dinosaur names, you will build a character level language model to generate new names. Your algorithm will learn the different name patterns, and randomly generate new names. Hopefully this algorithm will keep you and your team safe from the dinosaurs' wrath! 

By completing this assignment you will learn:

- How to store text data for processing using an RNN 
- How to synthesize data, by sampling predictions at each time step and passing it to the next RNN-cell unit
- How to build a character-level text generation recurrent neural network


In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

## 1 - Problem Statement

### 1.1 - Dataset and Preprocessing

Run the following cell to read the dataset of dinosaur names `dinos.txt`, create a list of unique characters (such as a-z), and compute the dataset and vocabulary size.

The characters are a-z (26 characters) plus the "\n" (or newline character), which in this assignment plays a role similar to the `<EOS>` (or "End of sentence") token. It indicates the end of the dinosaur name.

**TO DO**: In the cell below, you need to create two python dictionaries (i.e., hash tables) to map each character to an index from 0-26 and to map each index back to the corresponding character. 

This will help you figure out what index corresponds to what character in the probability distribution output of the softmax layer. Below, `self.ch_to_idx` and `self.idx_to_ch` are the python dictionaries. 

`__getitem__` is essential when using `DataLoader` later in the algorithm. The first entry of `x_str` being ` ` will be interpreted by setting $x^{\langle 0 \rangle} = \vec{0}$. Further, this ensures that `y` is equal to `x` but shifted one step to the left, and with an additional "\n" appended to signify the end of the dinosaur name. 

In [2]:
hidden_size = 100

class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('G:\pythonalgos\RNNhw\dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) #num of character (27 in this case)
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        # TO DO
        # YOUR CODE HERE
        self.ch_to_idx = {}
        self.idx_to_ch = {}
        for i in range(97,97+26):
            temp_i = i-96
            self.ch_to_idx[chr(i)] = temp_i
            self.idx_to_ch[temp_i] = chr(i)
        self.ch_to_idx["\n"] = 0
        self.idx_to_ch[0] = "\n"
    
    def __getitem__(self, index):
        line = self.lines[index]
        x_str = ' ' + line # add a space at the beginning, which indicates a vector of zeros.
        y_str = line + '\n'
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        
        y[0] = self.ch_to_idx[y_str[0]]
        #we start from the second character because the first character of x was nothing(vector of zeros).
        for i, (x_ch, y_ch) in enumerate(zip(x_str[1:], y_str[1:]), 1):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [3]:
d = DinosDataset()
print(d.vocab)
print(d.idx_to_ch.keys())

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0])


### 1.2 - Overview of the Recurrent Neural Network

**TO DO**: Implement your RNN model class. Your RNN model will have the following structure: 
    
<img src="images/RNN.png" style="width:450;height:300px;">
<caption><center> **Figure 1**: Recurrent Neural Network.  </center></caption>

At each time-step, the RNN tries to predict what is the next character given the previous characters. The dataset $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ is a list of characters in the training set, while $Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ , such that at every time-step $t$, we have $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$. 

In [4]:
class RNN(nn.Module):
    def __init__(self,hidden_dim,input_dim=27,output_dim=27):
        super().__init__()
        # TO DO
        # YOUR CODE HERE
        #in this case the dimension of x^(i) is 27, the dimension of a^(i) is defined by user
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(input_dim+hidden_dim, hidden_dim) #w_ax
        self.fc2 = nn.Linear(input_dim+hidden_dim, output_dim) #w_aa
        
    def forward(self,x0,a0):
        # TO DO
        # YOUR CODE HERE
        combined = torch.cat((x0, a0), 1)
        #combined = torch.squeeze(combined)
        #print("com:",combined.shape)
        a = self.fc1(combined)
        y = self.fc2(combined)       
        return a, y

In [5]:
n_hidden = 128
trn_ds = RNN(n_hidden)

## 2 - Sampling

In this part, you will build the important block of the overall language model:
- Sampling: a technique used to generate characters


Now assume that your model is trained. You would like to generate new text (characters). The process of generation is explained in the picture below:

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center> **Figure 2**: In this picture, we assume the model is already trained. We pass in $x^{\langle 1\rangle} = \vec{0}$ at the first time step, and have the network then sample one character at a time. </center></caption>

**TO DO**: Implement the `sample` function below to sample characters. You need to carry out 4 steps:

- **Step 1**: Pass the network the first "dummy" input $x^{\langle 1 \rangle} = \vec{0}$ (the vector of zeros). This is the default input before we've generated any characters. We also set $a^{\langle 0 \rangle} = \vec{0}$

- **Step 2**: Run one step of forward propagation to get $a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$. Here are the equations:

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

Note that $\hat{y}^{\langle t+1 \rangle }$ is a (softmax) probability vector (its entries are between 0 and 1 and sum to 1). $\hat{y}^{\langle t+1 \rangle}_i$ represents the probability that the character indexed by "i".

- **Step 3**: Carry out sampling: Pick the next character's index according to the probability distribution specified by $\hat{y}^{\langle t+1 \rangle }$. This means that if $\hat{y}^{\langle t+1 \rangle }_i = 0.16$, you will pick the index "i" with 16% probability. To implement it, you can use [`np.random.choice`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html).

Here is an example of how to use `np.random.choice()`:
```python
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3], p = p.ravel())
```
This means that you will pick the `index` according to the distribution: 
$P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.

- **Step 4**: The last step to implement in `sample()` is to overwrite the variable `x`, which currently stores $x^{\langle t \rangle }$, with the value of $x^{\langle t + 1 \rangle }$. You will represent $x^{\langle t + 1 \rangle }$ by creating a one-hot vector corresponding to the character you've chosen as your prediction. You will then forward propagate $x^{\langle t + 1 \rangle }$ in Step 1 and keep repeating the process until you get a "\n" character, indicating you've reached the end of the dinosaur name. 

In [6]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['\n']
    indices = []
    pred_char_idx = -1
    hidden_dim = model.hidden_dim
    input_dim = 27
    
    # Step 1: initialize first input and hidden state
    # YOUR CODE HERE
    h_prev = torch.zeros([1,hidden_dim], dtype=torch.float) #same as a^(i)
    x = torch.zeros(1,input_dim)
    
    with torch.no_grad():
        while pred_char_idx != newline_idx and word_size != 50:
            # Step 2: Forward propagate x using the equations (1), (2) and (3)
            # YOUR CODE HERE
            h_prev,y = model.forward(x,h_prev)
            #print("y:",y)
            y_hat = F.softmax(y)
            #print("y_hat1:",y_hat)
            y_hat = torch.squeeze(y_hat)
            #print("y_hat2:",y_hat)
            y_hatnp = y_hat.numpy()
            #if word_size == 2:
                #print(y_hatnp)
            #print(sum(y_hatnp))
    
            np.random.seed(np.random.randint(1, 5000))
            # Step 3: Sample the index of a character within the vocabulary from the probability distribution
            # YOUR CODE HERE
            idx = np.random.choice(27,1,p=y_hatnp)[0]
            indices.append(idx)
            
            # Step 4: Overwrite the input character as the one corresponding to the sampled index.
            # YOUR CODE HERE
            x = torch.zeros(1,input_dim)
            x[0][idx] = 1
            
            pred_char_idx = idx 
            word_size += 1
        if word_size == 50:
            indices.append(newline_idx)
    return indices

def print_sample(sample_idxs):
    print(trn_ds.idx_to_ch[sample_idxs[0]].upper(), end='')
    [print(trn_ds.idx_to_ch[x], end='') for x in sample_idxs[1:]]

## 3 - Training the language model 

It is time to train the character-level language model for text generation. 

### 3.1 - Gradient descent 

**TO DO**: In this section you will implement a function performing one epoch of training steps(with clipped gradients). You will go through the training examples one at a time, so the optimization algorithm will be stochastic gradient descent. 
Before running the optimization loop, you need to first initialize parameters.

As a reminder, here are the steps of a common optimization loop for an RNN:

- Forward propagate through the RNN to compute the loss
- Backward propagate through time to compute the gradients of the loss with respect to the parameters
- Clip the gradients if necessary 
- Update your parameters using gradient descent 

Every 100 steps of stochastic gradient descent, you will sample 1 name to see how the algorithm is doing. 

In [7]:
def train_one_epoch(model, loss_fn, optimizer):
    # Go through the training examples one at a time
    
    hidden_dim = model.hidden_dim
    for line_num, (x, y) in enumerate(trn_dl):
        model.train()
        loss = 0
        optimizer.zero_grad()
        #print(x)
        #print(y)
        
        # Initialize parameters
        # YOUR CODE HERE
        h_prev = torch.zeros([1,hidden_dim], dtype=torch.float)
        
        for i in range(x.shape[1]):
            # Forward propagate through the RNN to compute the loss
            # YOUR CODE HERE
            h_prev,y_out = model.forward(x[:,i],h_prev)
            #print("h_prev:",h_prev)
            #print("y_out",y_out)
            y_hat = F.softmax(y_out)
            #print("y_hat",y_hat)
            #print("y_hat type:",type(y_hat[0][0]))
            y_old = torch.zeros([1,27], dtype=torch.long)
            #print("y_old",y_old)
            #print("y_old type:",type(y_old[0][0]))
            y_old[0][y[0][i]] = 1
            if i == 0:
                y_ori = y_old
                y_h = y_hat
            else:
                y_ori = torch.cat((y_ori, y_old), 0)
                y_h = torch.cat((y_h, y_hat), 0)
        #print("y_ori",y_ori)
        #print("y_h",y_h)
        targetnp=y_ori.numpy()
        idxs=np.where(targetnp>0)[1]
        new_targets=torch.LongTensor(idxs)
        loss = loss_fn(y_h,new_targets)
            
        #Every 100 steps of stochastic gradient descent, print one sampled name to see how the algorithm is doing
        if (line_num+1) % 100 == 0:
            # YOUR CODE HERE
            # HINT: print_sample()
            indices = sample(model)
            print_sample(indices)
            print(loss)
        
        # Backpropagate through time
        # YOUR CODE HERE
        loss.backward()
        
        # Clip your gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        # Update parameters
        # YOUR CODE HERE
        optimizer.step()
        
        

### 3.2 - Begin Training
Remember to shuffle the dataset, so that stochastic gradient descent visits the examples in random order. 

**TO DO**: Follow the instructions and implement `train()`. 

In [8]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True)

def train(trn_ds, trn_dl, epochs=1):
    # Create a new model, loss_fn and optimizer.
    # YOUR CODE HERE
    model = RNN(128)
    # Use cross entropy loss
    loss_fn = nn.CrossEntropyLoss()
    # Use Adam
    optimizer = optim.Adam(model.parameters(),lr=1e-3)
    
    for e in range(1, epochs+1):
        print(f'{"-"*20} Epoch {e} {"-"*20}')
        train_one_epoch(model, loss_fn, optimizer)

In [9]:
#Start training
#When first run it, perhaps you will get wield result. I don't know why this happens.
#If so, run this block again and it works.
train(trn_ds, trn_dl, epochs=5)

-------------------- Epoch 1 --------------------


g:\pythonalgos\venv\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


g:\pythonalgos\venv\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Grahsnuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu
tensor(3.2129, grad_fn=<NllLossBackward>)


Xoehrnaouussssusussassssusssssssssssssssssssssssss
tensor(3.2886, grad_fn=<NllLossBackward>)


...

Raaoosaurus
tensor(3.2087, grad_fn=<NllLossBackward>)


Uanasaurus
tensor(3.0679, grad_fn=<NllLossBackward>)


Losaurus
tensor(3.1258, grad_fn=<NllLossBackward>)


Lanonosaurus
tensor(3.1392, grad_fn=<NllLossBackward>)


Manosaurus
tensor(3.0085, grad_fn=<NllLossBackward>)


## Conclusion

You can see that your algorithm has started to generate plausible dinosaur names towards the end of the training. At first, it was generating random characters, but towards the end you could see dinosaur names with cool endings. Feel free to run the algorithm even longer and play with hyperparameters to see if you can get even better results. Our implemetation generated some really cool names like `maconucon`, `marloralus` and `macingsersaurus`. Your model hopefully also learned that dinosaur names tend to end in `saurus`, `don`, `aura`, `tor`, etc.

If your model generates some non-cool names, don't blame the model entirely--not all actual dinosaur names sound cool. (For example, `dromaeosauroides` is an actual dinosaur name and is in the training set.) But this model should give you a set of candidates from which you can pick the coolest! 

This assignment had used a relatively small dataset, so that you could train an RNN quickly on a CPU. Training a model of the english language requires a much bigger dataset, and usually needs much more computation, and could run for many hours on GPUs. We ran our dinosaur name for quite some time, and so far our favoriate name is the great, undefeatable, and fierce: Mangosaurus!

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">

Reference: this assignment is adapted from one of Andrew Ng's Deep Learning Specialization--Sequence Models labs.